In [ ]:
!pip install arize-phoenix openinference-instrumentation-dspy opentelemetry-exporter-otlp

In [ ]:
!pip install "dspy-ai[mongodb]" cohere

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import phoenix as px

px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
import dspy
import dsp
import cohere
import json
from dotenv import load_dotenv
import os
import requests
from datetime import datetime
import backtrader as bt
load_dotenv('/teamspace/studios/this_studio/sentiment_analysis/.env')

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [4]:
def init_cerebro_object(strategy, list_of_data, stake = 100, cash=20000):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    for data in list_of_data:
        cerebro.adddata(data)
    cerebro.broker.set_cash(cash)
    return cerebro

def run_cerebro(strategy, list_data, stake=100, cash=20000):
    cerebro = init_cerebro_object(strategy, list_data, stake, cash)
    thestrats = cerebro.run()
    return cerebro, thestrats


def get_code_from_text(text):
    """Extracts the Python code segment from the provided text."""
    print("Text",text)
    code_segment = text.split("```")[1]
    if "```python" in text:
        code_segment = code_segment[6:]
    return code_segment.strip()

def load_base_strategy():
    """Loads the base strategy code from the base_strategy.py file."""
    with open("base_strategy.py", "r") as f:
        base_strats = "\n".join(f.readlines())
    return base_strats

def extract_error_message(error):
    error_lines = str(error).split('\n')
    for line in error_lines:
        if 'Error' in line or 'Exception' in line:
            return line.strip()
    return error_lines[-1].strip()

In [5]:
base_strats = load_base_strategy()

prompt = "Please create a strategy that buys when the price moves a certain percentage above the ATR value and sells when the price moves the same percentage below the ATR value"

instruction = f"""
    You are a python developer that intent to make a workable strategy from human requirement.
    Your task is to create a new BackTestStrategy that follow below msg
    Note: Only return class strategy and change `execute` function only, nothing else.
       
    And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function.
    Please name your class strategy as BackTestStrategy
    ----------------------------
    "{base_strats}"
    """

In [6]:
# Get example
import pandas as pd
from dspy.datasets.dataset import Dataset

# Define a simple signature for basic question answering
# Define a signature for generating a specific finance strategy from Python code
class FinanceStrategyGenerator(dspy.Signature):
    """Generate Python code for a specified finance strategy. """
    instruction = dspy.InputField(desc="Code structure and instruction for the finance strategy.")
    query = dspy.InputField(desc="Query of the finance strategy.")
    # generated_code = dspy.OutputField(desc="Generated Python code format for the strategy inside ```python``` block.")
    generated_code = dspy.OutputField(desc="Generated Python code for the strategy, formatted inside ```python``` block. If Backtrader is indicated, the code will inherit from Backtrader's Indicator class; otherwise, it will generate an LLM's knowledge indicator.")

    
class CSVDataset:
    def __init__(self, file_path, instruction=None) -> None:
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Change name of the columns
        df.columns = ['query', 'generated_code']
        
        instruction = instruction if instruction else "You are a data scientist that intent to make a workable strategy from human requirement"
        
        df['instruction'] = [instruction] * len(df)
        df = df.sample(frac=1).reset_index(drop=True)
        
        
        self.train = self._change_input(df.iloc[:70].to_dict(orient='records'))
        self.dev = self._change_input(df.iloc[70:].to_dict(orient='records'))
        
    
    def _change_input(self, input_data):
        
        ds = []
        for d in input_data:
            ds.append(
                dspy.Example(
                    instruction=d['instruction'],
                    query=d['query'],
                    generated_code=d['generated_code']
                ).with_inputs("instruction", "query")
            )
        
        return ds
    

file_path = "/teamspace/studios/this_studio/Data/complex_trading_strategies.csv"
dataset = CSVDataset(file_path=file_path, instruction=instruction)

In [7]:
def check_valid_code(strategy, list_data):
    try:
        cerebro, thestrats = run_cerebro(strategy, list_data)
        return True, ""
    except Exception as e:
        error_message = extract_error_message(e)
        return False, error_message


class GenerateCodeWithAssert(dspy.Module):
  def __init__(self, list_ohcl_data):
    super().__init__()
    self.generate_result = dspy.ChainOfThought(FinanceStrategyGenerator)
    self.ohcl_data = list_ohcl_data
  
  def forward(self, input: dspy.Example):
    ex = self.generate_result(instruction=input.instruction, query=input.query)
    exec(get_code_from_text(ex.generated_code), globals())
    check, error = check_valid_code(BackTestStrategy, self.ohcl_data)
    # dspy.Assert(check, f"Fix error {error}")

    dspy.Suggest(check, f"The code must not obtain the error {error}")

    return ex

In [12]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from ver_3.data_loader import load_stock_data
from ver_3.base_strategy import BaseStrategy
from dspy.predict import Retry


lm = dspy.Anyscale(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    max_tokens=512, 
    use_chat_api=True
)

dspy.settings.configure(lm=lm, trace=[])

data = [bt.feeds.PandasData(
                dataname=load_stock_data(ticker='AAPL', period="1y"), datetime="Date", 
                timeframe=bt.TimeFrame.Minutes)]

generate_with_assert = assert_transform_module(GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), backtrack_handler)

example = generate_with_assert(dataset.train[30])

print(f"Question: {prompt}")
print(f"Final Predicted Answer (after CoT process): {example.generated_code}")

[*********************100%%**********************]  1 of 1 completed


ERROR [dspy.primitives.assertions] 2024-06-16T01:35:41.418970Z [error    ] SuggestionFailed: The code must not obtain the error UltimateOscillator.__init__() got an unexpected keyword argument 'period1' [dspy.primitives.assertions] filename=assertions.py lineno=111


Text ```python
class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.uo = bt.ind.UltimateOscillator(self.data.close, period1=7, period2=14, period3=28)

    def execute(self):
        if self.uo > 50 and self.uo[-1] <= 50:
            return 1
        elif self.uo < 50 and self.uo[-1] >= 50:
            return -1
        else:
            return 0
```


ERROR [dspy.primitives.assertions] 2024-06-16T01:35:46.099443Z [error    ] SuggestionFailed: The code must not obtain the error UltimateOscillator.__init__() got an unexpected keyword argument 'period1' [dspy.primitives.assertions] filename=assertions.py lineno=111


Text ```python
class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.uo = bt.ind.UltimateOscillator(self.data, period1=7, period2=14, period3=28)
    def execute(self):
        if self.uo > 50 and self.uo[-1] <= 50:
            return 1
        elif self.uo < 50 and self.uo[-1] >= 50:
            return -1
        else:
            return 0
```


ERROR [dspy.primitives.assertions] 2024-06-16T01:35:52.007924Z [error    ] SuggestionFailed: The code must not obtain the error UltimateOscillator.__init__() got an unexpected keyword argument 'period1' [dspy.primitives.assertions] filename=assertions.py lineno=108


Text ```python
class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.uo = bt.ind.UltimateOscillator(self.data, period1=7, period2=14, period3=28)
    def execute(self):
        if self.uo[0] > 50 and self.uo[-1] <= 50:
            return 1
        elif self.uo[0] < 50 and self.uo[-1] >= 50:
            return -1
        else:
            return 0
```
Question: Please create a strategy that buys when the price moves a certain percentage above the ATR value and sells when the price moves the same percentage below the ATR value
Final Predicted Answer (after CoT process): ```python
class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.uo = bt.ind.UltimateOscillator(self.data, period1=7, period2=14, period3=28)
    def execute(self):
        if self.uo[0] > 50 and self.uo[-1] <= 50:
            return 1
        elif self.uo[0] < 50 a

In [10]:
lm.inspect_history(n=3)




Generate Python code for a specified finance strategy.

---

Follow the following format.

Instruction: Code structure and instruction for the finance strategy.

Query: Query of the finance strategy.

Reasoning: Let's think step by step in order to ${produce the generated_code}. We ...

Generated Code: Generated Python code for the strategy, formatted inside ```python``` block. If Backtrader is indicated, the code will inherit from Backtrader's Indicator class; otherwise, it will generate an LLM's knowledge indicator.

---

Instruction: You are a python developer that intent to make a workable strategy from human requirement. Your task is to create a new BackTestStrategy that follow below msg Note: Only return class strategy and change `execute` function only, nothing else. And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function. Please name your class strategy as BackTestStrategy -

'\n\n\nGenerate Python code for a specified finance strategy.\n\n---\n\nFollow the following format.\n\nInstruction: Code structure and instruction for the finance strategy.\n\nQuery: Query of the finance strategy.\n\nReasoning: Let\'s think step by step in order to ${produce the generated_code}. We ...\n\nGenerated Code: Generated Python code for the strategy, formatted inside ```python``` block. If Backtrader is indicated, the code will inherit from Backtrader\'s Indicator class; otherwise, it will generate an LLM\'s knowledge indicator.\n\n---\n\nInstruction: You are a python developer that intent to make a workable strategy from human requirement. Your task is to create a new BackTestStrategy that follow below msg Note: Only return class strategy and change `execute` function only, nothing else. And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function. Please name your class strateg

In [11]:
import matplotlib
%matplotlib inline
cerebro, thestrats = run_cerebro(strategy=BackTestStrategy, list_data=data, stake=100, cash=1000)
    #thestrat = thestrats[0]

print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
print("Total point return: ", (cerebro.broker.getvalue() - cerebro.broker.startingcash))

# Plot the results
figs = cerebro.plot(
    iplot=False, 
    style="pincandle", width=60 * 10, height=40 * 10
)


TypeError: UltimateOscillator.__init__() got an unexpected keyword argument 'period1'

# Finetuning prompt 

In [28]:
dataset.train[0]

Example({'instruction': '\n    You are a python developer that intent to make a workable strategy from human requirement.\n    Your task is to create a new BackTestStrategy that follow below msg\n    Note: Only return class strategy and change `execute` function only, nothing else.\n       \n    And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function.\n    Please name your class strategy as BackTestStrategy\n    ----------------------------\n    "import backtrader as bt\n\n\n\n\n\nclass BaseStrategy(bt.Strategy):\n\n    def __init__(self, debug=True):\n\n        """\n\n        final signal: 1 -> long, 0 -> neutral ->, -1 sell\n\n        """\n\n        # To keep track of pending orders and buy price/commission\n\n        self.countBuy = 0\n\n        self.countSell = 0\n\n        self.final_signal = None\n\n        self.debug = debug\n\n\n\n    def log(self, txt, dt=None):\n\n        """

In [20]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate.metrics import answer_exact_match

def validate_answer(example, pred, trace=None):
    return example.generated_code.lower() == pred.generated_code.lower()

teleprompter = BootstrapFewShotWithRandomSearch(metric = validate_answer, max_bootstrapped_demos=2, num_candidate_programs=6)

generated_code_student_teacher = teleprompter.compile(
                student=assert_transform_module(GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), backtrack_handler), 
                teacher = assert_transform_module(GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), backtrack_handler), 
                trainset=dataset.train, 
                valset=dataset.dev
)


# evaluate(cited_longformqa_student_teacher)

ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.048042Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149
ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.050397Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149


3
3
3
3
3
3
3


Average Metric: 0.0 / 2  (0.0):   3%|▎         | 1/30 [00:00<00:00, 382.48it/s]ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.051527Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149
ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.053206Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149
Average Metric: 0.0 / 4  (0.0):  10%|█         | 3/30 [00:00<00:00, 298.45it/s]ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.053434Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149
ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50.054020Z [error    ] Error for example in dev set: 		 missing a required argument: 'input' [dspy.evaluate.evaluate] filename=evaluate.py lineno=149
ERROR [dspy.evaluate.evaluate] 2024-06-15T16:28:50

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


TypeError: missing a required argument: 'input'

In [ ]:

# Save the generated code module
generated_code_student_teacher.save("generated_code.json")